In [55]:
import numpy as np
import scipy.stats as st

def up_and_in_call_mc(S0, K, B, r, sigma, T, N, M):
    dt = T / N
    discount = np.exp(-r * T)

    dW = np.sqrt(dt) * np.random.randn(M, N) 
    S = np.full((M, N + 1), S0)
    
    for t in range(1, N + 1):
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, t - 1])

    S_max = np.max(S, axis=1) 
    payoffs = discount * np.maximum(S[:, -1] - K, 0) * (S_max > B)  

    price = np.mean(payoffs)
    std_err = np.std(payoffs, ddof=1) / np.sqrt(M) 
    ci = st.t.interval(0.95, df=M-1, loc=price, scale=std_err) 

    return price, ci


def up_and_in_call_mc_antithetic(S0, K, B, r, sigma, T, N, M):

    dt = T / N
    discount = np.exp(-r * T)

    dW = np.sqrt(dt) * np.random.randn(M, N) 
    dW_antithetic = -dW  
    S1 = np.full((M, N + 1), S0)
    S2 = np.full((M, N + 1), S0)
    
    for t in range(1, N + 1):
        S1[:, t] = S1[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW[:, t - 1])
        S2[:, t] = S2[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * dW_antithetic[:, t - 1])

    S_max1, S_max2 = np.max(S1, axis=1), np.max(S2, axis=1)
    payoffs1 = discount * np.maximum(S1[:, -1] - K, 0) * (S_max1 > B)  
    payoffs2 = discount * np.maximum(S2[:, -1] - K, 0) * (S_max2 > B) 

    payoffs = 0.5 * (payoffs1 + payoffs2)

    price = np.mean(payoffs)
    std_err = np.std(payoffs, ddof=1) / np.sqrt(2 * M) 
    ci = st.t.interval(0.95, df=2*M-1, loc=price, scale=std_err) 

    return price, ci





In [59]:
S0 = 100
K = 110
B = 120
r = 0.05
sigma = 0.45
T = 1
N = 252

M_values = [10000, 50000, 100000,1000000]

for M in M_values:
    print(f"\n=== Results for M = {M} ===")
    
    price, ci = up_and_in_call_mc(S0, K, B, r, sigma, T, N, M)
    print(f"Option price: {price:.4f}")
    print(f"95% CI: ({ci[0]:.4f}, {ci[1]:.4f})")
    print(f"N sim: {N}")

    price_anti, ci_anti = up_and_in_call_mc_antithetic(S0, K, B, r, sigma, T, N, M)
    print(f"Option price (Antithetic): {price_anti:.4f}")
    print(f"95% CI (Antithetic): ({ci_anti[0]:.4f}, {ci_anti[1]:.4f})")



=== Results for M = 10000 ===
Option price: 0.0586
95% CI: (0.0220, 0.0952)
N sim: 252
Option price (Antithetic): 0.0644
95% CI (Antithetic): (0.0461, 0.0826)

=== Results for M = 50000 ===
Option price: 0.0635
95% CI: (0.0448, 0.0822)
N sim: 252
Option price (Antithetic): 0.0593
95% CI (Antithetic): (0.0507, 0.0679)

=== Results for M = 100000 ===
Option price: 0.0576
95% CI: (0.0451, 0.0702)
N sim: 252
Option price (Antithetic): 0.0602
95% CI (Antithetic): (0.0543, 0.0661)

=== Results for M = 1000000 ===
Option price: 0.0593
95% CI: (0.0554, 0.0632)
N sim: 252
Option price (Antithetic): 0.0583
95% CI (Antithetic): (0.0564, 0.0602)
